In [1]:
import numpy as np
import os
import argparse
import traceback
import time
import h5py
import sqlite3
import sys

from sotodlib import core
from sotodlib.core.flagman import has_all_cut, has_any_cuts, count_cuts
import sotodlib.site_pipeline.util as sp_util
from sotodlib.preprocess import preprocess_util
from so3g.proj import Ranges, RangesMatrix

# Example of Querying Stored Data

In [12]:
sqlite_path = '/scratch/gpfs/SIMONSOBS/users/ms3067/iso_stats/v1/20250718_cut_stats_satp1.sqlite'
conn = sqlite3.connect(sqlite_path)
cur = conn.cursor()

cur.execute('PRAGMA table_info(results)')
keys = [row[1] for row in cur.fetchall()]

obs_id = 'obs_1734310353_satp1_1111111'
cur.execute('SELECT * FROM results WHERE obsid=?', (obs_id,))
row = cur.fetchone()
conn.close()

for key, val in zip(keys, row):
    if isinstance(val, bytes):
        # I'm skipping the stuff stored as bytes
        # I tried to store numpy arrays but didn't do the datatype conversion right
        continue
        # arr = np.frombuffer(val, dtype=np.float32)
        # print(f"{key}: {arr}")
    else:
        print(f"{key}: {val}")

obsid: obs_1734310353_satp1_1111111
ws: ws0
band: f090
nsamps: 527018
ndets: 714
fp_cuts: 0
trend_cuts: 0
turnaround_nsamps: 5302260
jumps_slow_nsamps: 0
jumps_slow_cuts: 0
jumps_2pi_nsamps: 1353
jumps_2pi_cuts: 0
glitch_nsamps: 174043
glitch_cuts: 0
det_bias_cuts: 140
ptp_cuts: 6
white_noise_cuts: 24
edge_nsamps: 4543978
inv_var_cuts: 23
TOD_stats_T_det_cut: 0
TOD_stats_Q_det_cut: 0
TOD_stats_U_det_cut: 0
noisy_subscans_nsamps: 29272558
noisy_subscans_cuts: 89
source_flags_nsamps: 0
source_flags_cuts: 0
end_yield: 405


# Detector Cuts

In [2]:
sqlite_path = '/scratch/gpfs/SIMONSOBS/users/ms3067/iso_stats/v1/20250718_cut_stats_satp1.sqlite'
start_ts = 1716177600  # <-- set your start timestamp here (as int)
end_ts = 1734315803    # <-- set your end timestamp here (as int)
conn = sqlite3.connect(sqlite_path)
cur = conn.cursor()

cur.execute('PRAGMA table_info(results)')
columns = [row[1] for row in cur.fetchall()]
cut_keys = [k for k in columns if 'cuts' in k]

# Build the query to get all cut columns and end_yield at once
query = f"""
SELECT {', '.join(cut_keys)}, ndets FROM results
WHERE CAST(substr(obsid, 5, 10) AS INTEGER) BETWEEN ? AND ?
"""
cur.execute(query, (start_ts, end_ts))

# Use zip(*cur) to transpose rows to columns, then cast all to int
columns = zip(*cur)
cut_sums = [sum(x for x in col) for col in columns]

# The last element is the sum of end_yield, the rest are the sums for each cut_key
*cut_sums, total_end_yield = cut_sums
sum_of_all_cuts = sum(cut_sums)

ratios = [cut_sum / total_end_yield if total_end_yield else float('nan') for cut_sum in cut_sums]
fractions = [cut_sum / sum_of_all_cuts if sum_of_all_cuts else float('nan') for cut_sum in cut_sums]

conn.close()

for key, cut_sum, ratio, fraction in zip(cut_keys, cut_sums, ratios, fractions):
    #print(f"{key}: sum={cut_sum}, sum/total_end_yield={ratio:.4f}, sum/sum_of_all_cuts={fraction:.4f}")
    print(f"{key}: Total Fraction={ratio*100:.3f}%, Cuts Fraction={fraction*100:.3f}%")

print("_"*60)
print(f"total_ndets: {total_end_yield}")
print(f"sum_of_all_cuts: {sum_of_all_cuts}")
print(f"Det Cut Fraction: {100*sum_of_all_cuts/total_end_yield:.2f}%")

fp_cuts: Total Fraction=0.000%, Cuts Fraction=0.000%
trend_cuts: Total Fraction=0.011%, Cuts Fraction=0.020%
jumps_slow_cuts: Total Fraction=0.009%, Cuts Fraction=0.017%
jumps_2pi_cuts: Total Fraction=0.062%, Cuts Fraction=0.113%
glitch_cuts: Total Fraction=0.000%, Cuts Fraction=0.000%
det_bias_cuts: Total Fraction=35.221%, Cuts Fraction=63.755%
ptp_cuts: Total Fraction=4.680%, Cuts Fraction=8.472%
white_noise_cuts: Total Fraction=7.031%, Cuts Fraction=12.727%
inv_var_cuts: Total Fraction=1.193%, Cuts Fraction=2.160%
noisy_subscans_cuts: Total Fraction=7.037%, Cuts Fraction=12.738%
source_flags_cuts: Total Fraction=0.000%, Cuts Fraction=0.000%
____________________________________________________________
total_ndets: 21370008
sum_of_all_cuts: 11805677
Det Cut Fraction: 55.24%


In [8]:
sqlite_path = '/scratch/gpfs/SIMONSOBS/users/ms3067/iso_stats/v1/20250718_cut_stats_satp3.sqlite'
start_ts = 1722484800  # <-- set your start timestamp here (as int)
end_ts = 1734315803    # <-- set your end timestamp here (as int)
conn = sqlite3.connect(sqlite_path)
cur = conn.cursor()

cur.execute('PRAGMA table_info(results)')
columns = [row[1] for row in cur.fetchall()]
cut_keys = [k for k in columns if 'cuts' in k]

# Build the query to get all cut columns and end_yield at once
query = f"""
SELECT {', '.join(cut_keys)}, ndets FROM results
WHERE CAST(substr(obsid, 5, 10) AS INTEGER) BETWEEN ? AND ?
"""
cur.execute(query, (start_ts, end_ts))

# Use zip(*cur) to transpose rows to columns, then cast all to int
columns = zip(*cur)
cut_sums = [sum(x for x in col) for col in columns]

# The last element is the sum of end_yield, the rest are the sums for each cut_key
*cut_sums, total_end_yield = cut_sums
sum_of_all_cuts = sum(cut_sums)

ratios = [cut_sum / total_end_yield if total_end_yield else float('nan') for cut_sum in cut_sums]
fractions = [cut_sum / sum_of_all_cuts if sum_of_all_cuts else float('nan') for cut_sum in cut_sums]

conn.close()

for key, cut_sum, ratio, fraction in zip(cut_keys, cut_sums, ratios, fractions):
    #print(f"{key}: sum={cut_sum}, sum/total_end_yield={ratio:.4f}, sum/sum_of_all_cuts={fraction:.4f}")
    print(f"{key}: Total Fraction={ratio*100:.3f}%, Cuts Fraction={fraction*100:.3f}%")

print("_"*60)
print(f"total_ndets: {total_end_yield}")
print(f"sum_of_all_cuts: {sum_of_all_cuts}")
print(f"Det Cut Fraction: {100*sum_of_all_cuts/total_end_yield:.2f}%")

fp_cuts: Total Fraction=0.000%, Cuts Fraction=0.000%
trend_cuts: Total Fraction=0.028%, Cuts Fraction=0.115%
jumps_slow_cuts: Total Fraction=0.000%, Cuts Fraction=0.000%
jumps_2pi_cuts: Total Fraction=0.020%, Cuts Fraction=0.083%
glitch_cuts: Total Fraction=0.000%, Cuts Fraction=0.000%
det_bias_cuts: Total Fraction=15.941%, Cuts Fraction=66.149%
ptp_cuts: Total Fraction=2.013%, Cuts Fraction=8.353%
white_noise_cuts: Total Fraction=3.087%, Cuts Fraction=12.811%
inv_var_cuts: Total Fraction=1.158%, Cuts Fraction=4.804%
noisy_subscans_cuts: Total Fraction=1.852%, Cuts Fraction=7.686%
source_flags_cuts: Total Fraction=0.000%, Cuts Fraction=0.000%
____________________________________________________________
total_ndets: 14751680
sum_of_all_cuts: 3554860
Det Cut Fraction: 24.10%


# Sample Cuts

In [5]:
sqlite_path = '/scratch/gpfs/SIMONSOBS/users/ms3067/iso_stats/v1/20250718_cut_stats_satp1.sqlite'
start_ts = 1716177600
end_ts = 1734315803
conn = sqlite3.connect(sqlite_path)
cur = conn.cursor()

cur.execute('PRAGMA table_info(results)')
columns = [row[1] for row in cur.fetchall()]
cut_keys = [k for k in columns if ('nsamps' in k) & (k != 'nsamps')]

# Build the query to get all cut columns, nsamps, and end_yield at once
query = f"""
SELECT {', '.join(cut_keys)}, nsamps, end_yield FROM results
WHERE CAST(substr(obsid, 5, 10) AS INTEGER) BETWEEN ? AND ?
"""
cur.execute(query, (start_ts, end_ts))

# Fetch all rows
rows = list(cur)

# Transpose to columns for cuts, and keep nsamps and end_yield columns
cut_cols = list(zip(*rows))
cut_sums = [sum(int(x) for x in col) for col in cut_cols[:-2]]  # all cut_keys
nsamps_col = cut_cols[-2]
end_yield_col = cut_cols[-1]

# Compute total_end_yield as sum(nsamps * end_yield) over all rows
total_end_yield = sum(int(n) * int(e) for n, e in zip(nsamps_col, end_yield_col))
sum_of_all_cuts = sum(cut_sums)

ratios = [cut_sum / total_end_yield if total_end_yield else float('nan') for cut_sum in cut_sums]
fractions = [cut_sum / sum_of_all_cuts if sum_of_all_cuts else float('nan') for cut_sum in cut_sums]

conn.close()

for key, cut_sum, ratio, fraction in zip(cut_keys, cut_sums, ratios, fractions):
    print(f"{key}: Total Fraction={ratio*100:.3f}%, Cuts Fraction={fraction*100:.3f}%")

print("_"*60)
print(f"total_nsamps*end_yield: {total_end_yield}")
print(f"sum_of_all_samp_cuts: {sum_of_all_cuts}")
print(f"Samp Cut Fraction: {100*sum_of_all_cuts/total_end_yield:.2f}%")

turnaround_nsamps: Total Fraction=3.502%, Cuts Fraction=18.641%
jumps_slow_nsamps: Total Fraction=0.001%, Cuts Fraction=0.005%
jumps_2pi_nsamps: Total Fraction=0.000%, Cuts Fraction=0.003%
glitch_nsamps: Total Fraction=0.171%, Cuts Fraction=0.908%
edge_nsamps: Total Fraction=1.716%, Cuts Fraction=9.137%
noisy_subscans_nsamps: Total Fraction=13.394%, Cuts Fraction=71.305%
source_flags_nsamps: Total Fraction=0.000%, Cuts Fraction=0.001%
____________________________________________________________
total_nsamps*end_yield: 5799123481359
sum_of_all_samp_cuts: 1089338100622
Samp Cut Fraction: 18.78%


In [6]:
sqlite_path = '/scratch/gpfs/SIMONSOBS/users/ms3067/iso_stats/v1/20250718_cut_stats_satp3.sqlite'
start_ts = 1722484800  # <-- set your start timestamp here (as int)
end_ts = 1734315803    # <-- set your end timestamp here (as int)
conn = sqlite3.connect(sqlite_path)
cur = conn.cursor()

cur.execute('PRAGMA table_info(results)')
columns = [row[1] for row in cur.fetchall()]
cut_keys = [k for k in columns if ('nsamps' in k) & (k != 'nsamps')]

# Build the query to get all cut columns, nsamps, and end_yield at once
query = f"""
SELECT {', '.join(cut_keys)}, nsamps, end_yield FROM results
WHERE CAST(substr(obsid, 5, 10) AS INTEGER) BETWEEN ? AND ?
"""
cur.execute(query, (start_ts, end_ts))

# Fetch all rows
rows = list(cur)

# Transpose to columns for cuts, and keep nsamps and end_yield columns
cut_cols = list(zip(*rows))
cut_sums = [sum(int(x) for x in col) for col in cut_cols[:-2]]  # all cut_keys
nsamps_col = cut_cols[-2]
end_yield_col = cut_cols[-1]

# Compute total_end_yield as sum(nsamps * end_yield) over all rows
total_end_yield = sum(int(n) * int(e) for n, e in zip(nsamps_col, end_yield_col))
sum_of_all_cuts = sum(cut_sums)

ratios = [cut_sum / total_end_yield if total_end_yield else float('nan') for cut_sum in cut_sums]
fractions = [cut_sum / sum_of_all_cuts if sum_of_all_cuts else float('nan') for cut_sum in cut_sums]

conn.close()

for key, cut_sum, ratio, fraction in zip(cut_keys, cut_sums, ratios, fractions):
    print(f"{key}: Total Fraction={ratio*100:.3f}%, Cuts Fraction={fraction*100:.3f}%")

print("_"*60)
print(f"total_nsamps*end_yield: {total_end_yield}")
print(f"sum_of_all_samp_cuts: {sum_of_all_cuts}")
print(f"Samp Cut Fraction: {100*sum_of_all_cuts/total_end_yield:.2f}%")

turnaround_nsamps: Total Fraction=2.473%, Cuts Fraction=21.838%
jumps_slow_nsamps: Total Fraction=0.000%, Cuts Fraction=0.001%
jumps_2pi_nsamps: Total Fraction=0.000%, Cuts Fraction=0.004%
glitch_nsamps: Total Fraction=0.215%, Cuts Fraction=1.897%
edge_nsamps: Total Fraction=1.724%, Cuts Fraction=15.220%
noisy_subscans_nsamps: Total Fraction=6.913%, Cuts Fraction=61.040%
source_flags_nsamps: Total Fraction=0.000%, Cuts Fraction=0.000%
____________________________________________________________
total_nsamps*end_yield: 7114914515795
sum_of_all_samp_cuts: 805795747748
Samp Cut Fraction: 11.33%


# Finding Largest Yield Obs

In [14]:
sqlite_path = '/scratch/gpfs/SIMONSOBS/users/ms3067/iso_stats/v1/20250718_cut_stats_satp1.sqlite'
start_ts = 1716177600
end_ts = 1734315803
ws_value = 'ws0'  # Replace with the ws you want to select

conn = sqlite3.connect(sqlite_path)
cur = conn.cursor()

cur.execute(
    'SELECT obsid, band, end_yield FROM results WHERE ws=? ORDER BY end_yield DESC LIMIT 1',
    (ws_value,)
)
row = cur.fetchone()
if row:
    obsid, band, end_yield = row
    print(f"ws: {ws_value}, obsid: {obsid}, band: {band}, end_yield: {end_yield}")
else:
    print(f"No rows found for ws={ws_value}")

conn.close()

ws: ws0, obsid: obs_1730954780_satp1_1111111, band: f150, end_yield: 523


In [4]:
import sqlite3

sqlite_path = '/scratch/gpfs/SIMONSOBS/users/ms3067/iso_stats/v1/20250718_cut_stats_satp1.sqlite'
conn = sqlite3.connect(sqlite_path)
cur = conn.cursor()

# Find the maximum end_yield value
cur.execute('SELECT MAX(end_yield) FROM results')
max_end_yield = cur.fetchone()[0]

# Get all rows with that maximum end_yield
cur.execute('SELECT obsid, ws, band FROM results WHERE end_yield=?', (max_end_yield,))
rows = cur.fetchall()

print(f"Rows with maximum end_yield ({max_end_yield}):")
for obsid, ws, band in rows:
    print(f"obsid: {obsid}, ws: {ws}, band: {band}")

conn.close()

Rows with maximum end_yield (703):
obsid: obs_1718789308_satp1_1111111, ws: ws2, band: f150


In [1]:
import sqlite3

sqlite_path = '/scratch/gpfs/SIMONSOBS/users/ms3067/iso_stats/v1/20250718_cut_stats_satp3.sqlite'
conn = sqlite3.connect(sqlite_path)
cur = conn.cursor()

# Find the maximum end_yield value
cur.execute('SELECT MAX(end_yield) FROM results')
max_end_yield = cur.fetchone()[0]

# Get all rows with that maximum end_yield
cur.execute('SELECT obsid, ws, band FROM results WHERE end_yield=?', (max_end_yield,))
rows = cur.fetchall()

print(f"Rows with maximum end_yield ({max_end_yield}):")
for obsid, ws, band in rows:
    print(f"obsid: {obsid}, ws: {ws}, band: {band}")

conn.close()

Rows with maximum end_yield (774):
obsid: obs_1733534371_satp3_1111111, ws: ws6, band: f150
